In [11]:
import requests
from requests.auth import HTTPBasicAuth
import json
import urllib
import os
import time
import csv

In [12]:
#Make sure the env vars are set
api_token = os.environ['JIRA_API_TOKEN']
email_address = os.environ['JIRA_EMAIL_ADDRESS']

auth = HTTPBasicAuth(email_address, api_token)

headers = {
   "Accept": "application/json"
}

In [13]:
#URL encoded JQL
jql = urllib.parse.quote("project = SMM AND issuetype = Story AND status = Done order by created DESC")

In [14]:
max_results = 1000

In [15]:
#Retrieve all JIRA keys (e.g. SMM-1423) given a JQL
url_search = 'https://bulbenergy.atlassian.net/rest/api/3/search?maxResults={max_results}&fields=key&jql={jql}'.format(max_results = max_results, jql = jql)

In [16]:
search_response = requests.request(
   "GET",
   url_search,
   headers=headers,
   auth=auth
)

search_response = json.loads(search_response.text)

In [17]:
keys = [sr['key'] for sr in search_response['issues']]

#Line bellow is for test only
#keys = keys[:10]

In [20]:
#Changelog API endpoint - it needs to be issue by issue, I couldn't find a changelog API for bulk request
url = "https://bulbenergy.atlassian.net/rest/api/2/issue/{issue_key}/changelog"

In [21]:
def fetch_changelog(url, headers, auth, key):
    response = requests.request(
       "GET",
       url.format(issue_key = key),
       headers=headers,
       auth=auth
    )

    return json.loads(response.text).get('values')


#Extract the info required from the changelog of an issue
def extract_issue_stats(issue_key, issue_changes):
    ticket_stats = {'key': issue_key}
    
    for c in issue_changes:
        items = c.get('items', [])
        for i in items:
            if i.get('field', '') == 'status':
                ticket_stats[i.get('toString')] = c.get('created')
            elif i.get('field', '') == 'Story point estimate':
                ticket_stats['Estimation'] = i.get('toString')
    
    return ticket_stats

In [22]:
# Sleep between calls - not sure if this is necessary as we use JIRA Cloud but adding it as a precaution
sleep_secs = 0.3

tickets = []
for key in keys:
    changes = fetch_changelog(url, headers, auth, key)

    ticket_stats = extract_issue_stats(key, changes)
    
    tickets.append(ticket_stats)
    time.sleep(sleep_secs)


In [23]:
fieldnames = set()
for t in tickets:
    fieldnames.update(t.keys())
fieldnames = sorted(fieldnames)

csv_filename = os.path.expanduser("~") + '/' + "file_SMM__test.csv"
with open(csv_filename, "w", newline='') as fd:
    wr = csv.DictWriter(fd, fieldnames)
    wr.writeheader()
    wr.writerows(tickets)